In [ ]:
# access to google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# create a directory in drive
import os
print(os.getcwd())

In [ ]:
%cd "/content/gdrive/MyDrive//2023ECGChallenge"

### *load dataset*

In [ ]:
import numpy as np
import pandas as pd

x_train = pd.read_csv('x_train_lead_3.csv', index_col= 0)
y_train = pd.read_csv('y_train_lead_3.csv', index_col= 0)

# print(x_train.shape)
# print(y_train.shape)

In [ ]:
# mark accute/not accute
y_train = np.array(y_train['not_accute'])
y_train = y_train.reshape(-1, 1)

### *model creation*

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
from keras.layers import Input, Dense, Dropout, Activation, BatchNormalization, Add
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPool1D, ZeroPadding1D, LSTM, Bidirectional
from keras.models import Sequential, Model
from keras.utils import plot_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from scipy import optimize
from scipy.io import loadmat
import os

#### *a. Lenet*

In [ ]:
lenet_5_model=Sequential()

lenet_5_model.add(Conv1D(filters=6, kernel_size=3, padding='same', input_shape=(1000,1)))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))
lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

lenet_5_model.add(Conv1D(filters=16, strides=1, kernel_size=5))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))
lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

lenet_5_model.add(GlobalAveragePooling1D())

lenet_5_model.add(Dense(64, activation='relu'))

lenet_5_model.add(Dense(32, activation='relu'))

lenet_5_model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
import math

def lr_schedule(epoch):
    # define simple learning rate decay
    lr = 0.001 * math.pow(0.1, math.floor(epoch / 10))
    return lr

# print('Decaying Rate')
# print(lr_schedule(9), lr_schedule(10), lr_schedule(19), lr_schedule(20))

lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD, Adagrad

# compile model
lenet_5_model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

filepath = "best_model_resnet50.h5"
# define early_stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=30)
# define checkpoint
model_checkpoint = ModelCheckpoint(filepath, save_best_only=True, save_weights_only=False, monitor='val_loss', mode='min')

In [ ]:
# gpu usage
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

with tf.device(device_name):
    history = lenet_5_model.fit(x=x_train,
                          y=y_train,           
                          validation_split= 0.1,
                          batch_size=128,         
                          epochs=200,            
                          verbose=2,
                        callbacks=[lr_scheduler, early_stopping, model_checkpoint])

#### *b. VGG 16*

In [ ]:
vgg_16_model=Sequential()

vgg_16_model.add(Conv1D(filters=64, kernel_size=3, padding='same',  input_shape=(1000,1)))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=64, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=128, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=128, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=256, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=256, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=256, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=512, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=512, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=1, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=1, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(GlobalAveragePooling1D())
vgg_16_model.add(Dense(256, activation='relu'))
vgg_16_model.add(Dropout(0.4))
vgg_16_model.add(Dense(128, activation='relu'))
vgg_16_model.add(Dropout(0.4))
vgg_16_model.add(Dense(1, activation='sigmoid'))

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
import math

def lr_schedule(epoch):
    # define simple learning rate decay
    lr = 0.001 * math.pow(0.1, math.floor(epoch / 10))
    return lr

# print('Decaying Rate')
# print(lr_schedule(9), lr_schedule(10), lr_schedule(19), lr_schedule(20))

lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD, Adagrad

# compile model
vgg_16_model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

filepath = "best_model_vgg_16_model.h5"
# define early_stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=30)
# define checkpoint
model_checkpoint = ModelCheckpoint(filepath, save_best_only=True, save_weights_only=False, monitor='val_loss', mode='min')

In [ ]:
# gpu usage
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

with tf.device(device_name):
    history = vgg_16_model.fit(x=x_train,
                          y=y_train,           
                          validation_split= 0.1,
                          batch_size=128,         
                          epochs=200,            
                          verbose=2,
                        callbacks=[lr_scheduler, early_stopping, model_checkpoint])

#### *c. Inception*

In [ ]:
from tensorflow.keras.layers import concatenate

In [ ]:
def inception_block(prev_layer):
    
    conv1=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv1=BatchNormalization()(conv1)
    conv1=Activation('relu')(conv1)
    
    conv3=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv3=BatchNormalization()(conv3)
    conv3=Activation('relu')(conv3)
    conv3=Conv1D(filters = 64, kernel_size = 3, padding = 'same')(conv3)
    conv3=BatchNormalization()(conv3)
    conv3=Activation('relu')(conv3)
    
    conv5=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv5=BatchNormalization()(conv5)
    conv5=Activation('relu')(conv5)
    conv5=Conv1D(filters = 64, kernel_size = 5, padding = 'same')(conv5)
    conv5=BatchNormalization()(conv5)
    conv5=Activation('relu')(conv5)
    
    pool= MaxPool1D(pool_size=3, strides=1, padding='same')(prev_layer)
    convmax=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(pool)
    convmax=BatchNormalization()(convmax)
    convmax=Activation('relu')(convmax)
    
    layer_out = concatenate([conv1, conv3, conv5, convmax], axis=1)
    
    return layer_out

def inception_model(input_shape):
    X_input=Input(input_shape)
    
    X = ZeroPadding1D(3)(X_input)
    
    X = Conv1D(filters = 64, kernel_size = 7, padding = 'same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPool1D(pool_size=3, strides=2, padding='same')(X)
    
    X = Conv1D(filters = 64, kernel_size = 1, padding = 'same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    X = inception_block(X)
    X = inception_block(X)
    
    X = MaxPool1D(pool_size=7, strides=2, padding='same')(X)
    
    X = GlobalAveragePooling1D()(X)
    X = Dense(1,activation='sigmoid')(X)
    
    model = Model(inputs = X_input, outputs = X, name='Inception')
    
    return model

In [ ]:
inception_model = inception_model(input_shape = (1000,1))

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
import math

def lr_schedule(epoch):
    # define simple learning rate decay
    lr = 0.001 * math.pow(0.1, math.floor(epoch / 10))
    return lr

# print('Decaying Rate')
# print(lr_schedule(9), lr_schedule(10), lr_schedule(19), lr_schedule(20))

lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD, Adagrad

# compile model
inception_model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

filepath = "best_model_inception_model.h5"
# define early_stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=30)
# define checkpoint
model_checkpoint = ModelCheckpoint(filepath, save_best_only=True, save_weights_only=False, monitor='val_loss', mode='min')

In [ ]:
# gpu usage
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

with tf.device(device_name):
    history = inception_model.fit(x=x_train,
                          y=y_train,           
                          validation_split= 0.1,
                          batch_size=128,         
                          epochs=200,            
                          verbose=2,
                        callbacks=[lr_scheduler, early_stopping, model_checkpoint])

#### d. LSTM

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(1000,1), return_sequences=True))
lstm_model.add(LSTM(64))
lstm_model.add(Dense(32, activation = 'relu'))
lstm_model.add(Dropout(0.3))
lstm_model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
import math

def lr_schedule(epoch):
    # define simple learning rate decay
    lr = 0.001 * math.pow(0.1, math.floor(epoch / 10))
    return lr

# print('Decaying Rate')
# print(lr_schedule(9), lr_schedule(10), lr_schedule(19), lr_schedule(20))

lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD, Adagrad

# compile model
lstm_model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

filepath = "best_model_lstm_model.h5"
# define early_stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=30)
# define checkpoint
model_checkpoint = ModelCheckpoint(filepath, save_best_only=True, save_weights_only=False, monitor='val_loss', mode='min')

In [ ]:
# gpu usage
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

with tf.device(device_name):
    history = lstm_model.fit(x=x_train,
                          y=y_train,           
                          validation_split= 0.1,
                          batch_size=128,         
                          epochs=200,            
                          verbose=2,
                        callbacks=[lr_scheduler, early_stopping, model_checkpoint])

#### *e. GRU*

In [ ]:
from keras.layers import GRU

In [ ]:
GRU_model = Sequential()
GRU_model.add(GRU(64, input_shape=(1000,1), return_sequences=True))
GRU_model.add(GRU(64))
GRU_model.add(Dense(32, activation = 'relu'))
GRU_model.add(Dropout(0.3))
GRU_model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
import math

def lr_schedule(epoch):
    # define simple learning rate decay
    lr = 0.001 * math.pow(0.1, math.floor(epoch / 10))
    return lr

# print('Decaying Rate')
# print(lr_schedule(9), lr_schedule(10), lr_schedule(19), lr_schedule(20))

lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD, Adagrad

# compile model
GRU_model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

filepath = "best_model_GRU_model.h5"
# define early_stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=30)
# define checkpoint
model_checkpoint = ModelCheckpoint(filepath, save_best_only=True, save_weights_only=False, monitor='val_loss', mode='min')

In [ ]:
# gpu usage
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

with tf.device(device_name):
    history = GRU_model.fit(x=x_train,
                          y=y_train,           
                          validation_split= 0.1,
                          batch_size=128,         
                          epochs=200,            
                          verbose=2,
                        callbacks=[lr_scheduler, early_stopping, model_checkpoint])